In [1]:
import ee
import geemap

In [2]:
# 显示地图，并选中湖泊
Map = geemap.Map(center=[40.964,108.884],zoom = 10)
Map

Map(center=[40.964, 108.884], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [3]:
#在地图中选择研究区，并转为ROI。选定好后再运行下两行代码
Map.draw_last_feature
roi = ee.FeatureCollection(Map.draw_last_feature)

In [41]:
# 选择S2-TOA影像
s2_col = ee.ImageCollection('COPERNICUS/S2').filterBounds(roi)\
    .filter(ee.Filter.calendarRange(2019,2019,'year'))\
    .filter(ee.Filter.calendarRange(7,10,'month'))\

print(f"""已选择{s2_col.size().getInfo()}景S2影像""")
print(s2_col.aggregate_array('system:index').getInfo())
print(s2_col.aggregate_array('CLOUDY_PIXEL_PERCENTAGE').getInfo())




已选择49景S2影像
['20190701T033549_20190701T034354_T49TCF', '20190703T032541_20190703T033030_T49TCF', '20190706T033541_20190706T034532_T49TCF', '20190708T032549_20190708T033215_T49TCF', '20190711T033549_20190711T033859_T49TCF', '20190713T032541_20190713T033534_T49TCF', '20190716T033541_20190716T033833_T49TCF', '20190718T032549_20190718T032853_T49TCF', '20190721T033549_20190721T034527_T49TCF', '20190723T032541_20190723T033031_T49TCF', '20190726T033541_20190726T034456_T49TCF', '20190728T032549_20190728T032545_T49TCF', '20190731T033549_20190731T033843_T49TCF', '20190802T032541_20190802T033030_T49TCF', '20190805T033541_20190805T034618_T49TCF', '20190807T032549_20190807T033637_T49TCF', '20190810T033539_20190810T034635_T49TCF', '20190812T032541_20190812T033155_T49TCF', '20190815T033541_20190815T034352_T49TCF', '20190817T032539_20190817T033403_T49TCF', '20190820T033539_20190820T034220_T49TCF', '20190822T032541_20190822T032604_T49TCF', '20190825T033541_20190825T034232_T49TCF', '20190827T032539_20190

In [42]:
def s2_toa_select_by_cloud(s2_col,roi,threshold):
    ##依次传入有云影像集合、无云影像集合、研究区roi，云量阈值
    
    #计算withcloud_collection-Imagecollection中Image数量
    count = s2_col.size().getInfo()
    print(count)
    #定义图像裁剪函数
    def img_clip(image):
        clip_img = image.clip(roi)
        return clip_img        
    
    #进行图像裁剪
    s2_col_clip = s2_col.map(img_clip)
    s2_col_clip_list = s2_col_clip.toList(count)
    #创建列表，用来存储满足
    nice_list = []

    def rmCloud(image):
        qa = image.select('QA60')
        cloudBitMask = 1 << 10
        cirrusBitMask = 1 << 11
        mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
                .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
        return mask

    
    #定义云量计算函数
    def s2_img_cloud_cal(image):
        mask = rmCloud(image)
        hist = mask.reduceRegion(ee.Reducer.histogram(10), roi, 10).getInfo()
        if len(hist['QA60']['histogram']) == 2:
            count_clear = hist['QA60']['histogram'][1]
            count_sum = hist['QA60']['histogram'][0]+hist['QA60']['histogram'][1]
            print(f"""{image.getInfo()['properties']['system:index']}研究区云量为：{1-(count_clear/count_sum)},全景云量为:{image.getInfo()['properties']['CLOUDY_PIXEL_PERCENTAGE']/100},
                被选中：{(1-(count_clear/count_sum))<threshold}""")
            if((1-(count_clear/count_sum))<threshold):
                nice_list.append('COPERNICUS/S2/'+image.getInfo()['properties']['system:index'])
        elif len(hist['QA60']['histogram']) == 1:
            count_sum = hist['QA60']['histogram'][0]
#             print(f"""
#                 {image.getInfo()['properties']['system:index']}研究区云量为：{0},全景云量为:{image.getInfo()['properties']['CLOUDY_PIXEL_PERCENTAGE']/100},
#                 被选中：{0<threshold}""")
            nice_list.append('COPERNICUS/S2/'+image.getInfo()['properties']['system:index'])
    
    #遍历Image，计算云量，获取满足条件的影像。
    for i in range(count):
        img = ee.Image(s2_col_clip_list.get(i))
        print(f"""第{i+1}景影像处理中，未提示为研究区内无云""")
        s2_img_cloud_cal(img)
    
    #将选择的影像与少云影像集合合并
    selected_col = ee.ImageCollection(nice_list)
    
    return selected_col
    
    

In [43]:
result_col = s2_toa_select_by_cloud(s2_col,roi,0.1)

print(f"""共{result_col.size().getInfo()}景s2研究区无云影像""")
print(result_col.aggregate_array('system:index').getInfo())

49
第1景影像处理中，未提示为研究区内无云
第2景影像处理中，未提示为研究区内无云
第3景影像处理中，未提示为研究区内无云
第4景影像处理中，未提示为研究区内无云
20190708T032549_20190708T033215_T49TCF研究区云量为：0.05441538810318203,全景云量为:0.9831740000000001,
                被选中：True
第5景影像处理中，未提示为研究区内无云
20190711T033549_20190711T033859_T49TCF研究区云量为：0.04317499322587037,全景云量为:0.165392,
                被选中：True
第6景影像处理中，未提示为研究区内无云
第7景影像处理中，未提示为研究区内无云
20190716T033541_20190716T033833_T49TCF研究区云量为：0.2072779691849832,全景云量为:0.9094450000000001,
                被选中：False
第8景影像处理中，未提示为研究区内无云
20190718T032549_20190718T032853_T49TCF研究区云量为：0.03762979235824737,全景云量为:0.775754,
                被选中：True
第9景影像处理中，未提示为研究区内无云
第10景影像处理中，未提示为研究区内无云
第11景影像处理中，未提示为研究区内无云
第12景影像处理中，未提示为研究区内无云
20190728T032549_20190728T032545_T49TCF研究区云量为：0.05265080675259581,全景云量为:0.892626,
                被选中：True
第13景影像处理中，未提示为研究区内无云
第14景影像处理中，未提示为研究区内无云
第15景影像处理中，未提示为研究区内无云
第16景影像处理中，未提示为研究区内无云
第17景影像处理中，未提示为研究区内无云
20190810T033539_20190810T034635_T49TCF研究区云量为：0.16649461936377363,全景云量为:0.354146,
                被选